In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

from sklearn.preprocessing import MinMaxScaler
from fastprogress import progress_bar

import os

torch.manual_seed(0) # fix seed

### Parameters Set based on Model Training Results

![Model](https://github.com/MK-ek11/stock_price_forecast_with_LSTM/blob/main/images/model_plot.png?raw=1)

In [ ]:
# Set Parameter
sequence_length = 7 
batch = 64
hidden_size = 100
num_epochs = 150

### Set device 

- use CUDA ref: <https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html#creating-models>

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Functions

In [ ]:
# Create function for getting index of a date
def start_date_index(year, month, dataset):
    start_date = f"{year}-{month}"
    index_start_date = dataset[dataset["Date"].str.contains(start_date, regex=False)].index[0]
    
    print(f"Index of Date: {index_start_date}\nStart Date: {dataset.iloc[index_start_date,0]}", end="\n"*2)
    return index_start_date

In [ ]:
# Create function for splitting dataset
def split_dataset(start_index, sequence_length, dataset):
    train_set = dataset.loc[0:start_index-1]
    test_set = dataset.loc[start_index-sequence_length:len(dataset)].reset_index(drop=True)
    return train_set, test_set

In [ ]:
# Create function for Training set Sliding Window
def sliding_window_train(sequence_length, input_data):
    train = []
    target = []

    for i in range(sequence_length, len(input_data)):
        train.append(input_data[i-sequence_length:i])
        target.append(input_data[i])

    train = np.array(train)
    target = np.array(target)
    
    print(f"Train shape: {train.shape}",
         end="\n"*2)
    print(f"Train:\nbatch size: {train.shape[0]}\nsequence length: {train.shape[1]} days\ninput size: {train.shape[2]}",
          end="\n"*3)
    print(f"Target shape: {target.shape}",
         end="\n"*2)    
    print(f"Target:\nbatch size: {target.shape[0]}\ninput size: {target.shape[1]}",
          end="\n"*2)

    return train, target

In [ ]:
# Create function for Testing set Sliding Window
def sliding_window_test(start_index, sequence_length, input_data_scale, input_data_nonscale, input_date):
    test = []
    actual = []
    date = []
    open_price= []

    for i in range(start_index, len(input_data_scale)):
        test.append(input_data_scale[i-sequence_length:i])
        actual.append(input_data_scale[i])
        date.append(input_date[i])
        open_price.append(input_data_nonscale[i])

    test = np.array(test)
    actual = np.array(actual)
    date = np.array(date)
    open_price = np.array(open_price)
    
    print(f"Test shape: {test.shape}",
         end="\n"*2)
    print(f"Test:\nbatch size: {test.shape[0]}\nsequence length: {test.shape[1]} days\ninput size: {test.shape[2]}",
          end="\n"*3)
    print(f"Actual shape: {actual.shape}",
         end="\n"*2)    
    print(f"Actual:\nbatch size: {actual.shape[0]}\ninput size: {actual.shape[1]}",
          end="\n"*2)      

    return test, actual, date, open_price

In [ ]:
# Create function for RMSE and MSE
def MSE_RMSE_loss_np(predicted, target):
    mse = (np.square(np.subtract(target,predicted))).mean()
    rmse = np.sqrt(mse)
    print(f"MSE Loss: {mse} \nRMSE Loss: {rmse}",end="\n"*2)
    return rmse, mse

In [ ]:
# Create function for R2 Coefficient of determination
def R2_np(predicted, target):
    SSR = np.square(np.subtract(target,predicted)).sum()
    SST = np.square(target-target.mean()).sum()
    R2 = (1-(SSR/SST))
    print(f"R2: {R2}",end="\n"*2)
    return R2

### Extract Data 

In [ ]:
# Extract data
dataset_og = pd.read_csv(r"dataset\data.csv")

In [ ]:
# Description for missing data and columns types
missing_data = dataset_og.isnull().sum()
dtypes_data = dataset_og.dtypes
summary_df = pd.DataFrame(data = {"Missing Data":missing_data, "Columns Types":dtypes_data})
summary_df

,Missing Data,Columns Types
Date,0,object
Open,0,float64
High,0,float64
Low,0,float64
Close,0,float64
Adj Close,0,float64
Volume,0,int64


### Split Data

In [ ]:
# Define the date to split train and test
year = "2023"
month = "01"
# sequence_length = 7 


# Get the index of the date
index_start_date = start_date_index(year, month, dataset_og)


# build the train and test dataset
traindata, testdata = split_dataset(index_start_date, sequence_length, dataset_og)

Index of Date: 5787
Start Date: 2023-01-03



# Training & Testing and Price Prediction

#### Feature Scaling Training Data

In [ ]:
# Min Max Scaling for features
# Feature Scaling
min_max = (0,1)
feature = "Open"

scale_open = MinMaxScaler(feature_range = min_max)
data_open = traindata[feature].values.reshape(-1, 1)
data_open_scaled = scale_open.fit_transform(data_open)
data_open_scaled.shape

(5787, 1)

#### Prepare Training Data with Sliding Window

In [ ]:
# Prepare data with sliding window

train, target = sliding_window_train(sequence_length, data_open_scaled)

Train shape: (5780, 7, 1)

Train:
batch size: 5780
sequence length: 7 days
input size: 1


Target shape: (5780, 1)

Target:
batch size: 5780
input size: 1



#### Prepare Training Data for Training

In [ ]:
# Set data to Tensor
train = Variable(torch.Tensor(np.array(train).reshape(-1,sequence_length,1)))
target = Variable(torch.Tensor(np.array(target).reshape(-1,1)))

print("train shape is:",train.size())
print("train label shape is:",target.size(), end="\n"*2)

train shape is: torch.Size([5780, 7, 1])
train label shape is: torch.Size([5780, 1])



In [ ]:
# Breakdown of the Training Dataset
print(f"Training Input into LSTM Tensor Shape: {train.size()}", end = "\n"*2)
print(f"Training Input Tensor: \nbatch size: {train.size(0)}\nsequence length: {train.size(1)}\ninput size: {train.size(2)}",
      end="\n"*3)

print(f"Target Input into LSTM Tensor Shape: {target.size()}", end = "\n"*2)
print(f"Target Input Tensor: \nbatch size: {target.size(0)} \ninput size: {target.size(1)}",
      end="\n"*3)

Training Input into LSTM Tensor Shape: torch.Size([5780, 7, 1])

Training Input Tensor: 
batch size: 5780
sequence length: 7
input size: 1


Target Input into LSTM Tensor Shape: torch.Size([5780, 1])

Target Input Tensor: 
batch size: 5780 
input size: 1




In [ ]:
# Convert dataset to batches
# batch = 64

dataloader = DataLoader(TensorDataset(train, target), shuffle=False, batch_size=batch)

### Define the LSTM Model

In [ ]:
# Model
class LSTM(nn.Module):
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        # LSTM
        self.lstm1 = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_size,
                            num_layers=num_layers, 
                            batch_first=True,
                           )   
        # Dropout Layer
        self.dropout1 = nn.Dropout(p=0.2)        
        
        # Last Fully Connected
        self.fc = nn.Linear(hidden_size, num_classes)

        
    def forward(self, x):
        h_01 = Variable(torch.zeros( self.num_layers, 
                                   x.size(0),  # <== this is the batch size
                                   self.hidden_size))
        
        c_01 = Variable(torch.zeros(self.num_layers, 
                                   x.size(0),  # <== this is the batch size
                                   self.hidden_size))  
        
        # Propagate input through LSTM
        output, (h_n1, c_n1) = self.lstm1(x, (h_01.to(device), c_01.to(device))) 
        dropout_out1 = self.dropout1(h_n1)
        
        h_n_flattened = dropout_out1.view(-1, self.hidden_size) # <= Flatten Tensor
        fc_out = self.fc(h_n_flattened) # <= FC layer
        
        return fc_out

#### Set Parameters

In [ ]:
# Setting Parameters of LSTM
num_classes = 1
input_size = train.size(2)
# hidden_size = 100
num_layers = 1

# Epochs and learning rate
# num_epochs = 50
learning_rate = 1e-3

# Input into LSTM
batch_size = train.size(0)
seq_length = train.size(1)

In [ ]:
# Initialize the LSTM model
lstm_model = LSTM(num_classes, input_size, hidden_size, num_layers)
lstm_model.to(device)  # <= Set model to CUDA device

LSTM(
  (lstm1): LSTM(1, 100, batch_first=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=100, out_features=1, bias=True)
)

### Train model

In [ ]:
# Set loss_function and Optimzer
loss_function = torch.nn.MSELoss().to(device)    # mean-squared error
optimizer = torch.optim.Adam(lstm_model.parameters(),
                             lr=learning_rate,
                             weight_decay=1e-5)

# Train the model
for epoch in progress_bar(range(num_epochs)): 
    
    total_loss = 0
    for batch_num, data in enumerate(dataloader):
        
        train_data, target_data = data
        
        lstm_model.train()

        # Zero gradients
        optimizer.zero_grad()

        # Predictions
        predict_outputs = lstm_model(train_data.to(device)) # <= Set training to CUDA device

        # Obtain the loss function
        loss = loss_function(predict_outputs, target_data.to(device))

        # backward propogation
        loss.backward()

        # gradient descent
        optimizer.step()

        # Calculate Loss
        total_loss += loss.item()

        if batch_num % 100==0:
            print(f"Epoch: {epoch}, Batch: {batch_num}, Loss: {loss.item()}, Total Loss: {total_loss}")

Epoch: 0, Batch: 0, Loss: 0.0002938677207566798, Total Loss: 0.0002938677207566798
Epoch: 1, Batch: 0, Loss: 0.27531397342681885, Total Loss: 0.27531397342681885
Epoch: 2, Batch: 0, Loss: 0.3270838260650635, Total Loss: 0.3270838260650635
Epoch: 3, Batch: 0, Loss: 0.031589023768901825, Total Loss: 0.031589023768901825
Epoch: 4, Batch: 0, Loss: 0.042469322681427, Total Loss: 0.042469322681427
Epoch: 5, Batch: 0, Loss: 0.0069220587611198425, Total Loss: 0.0069220587611198425
Epoch: 6, Batch: 0, Loss: 0.01040133647620678, Total Loss: 0.01040133647620678
Epoch: 7, Batch: 0, Loss: 0.014146313071250916, Total Loss: 0.014146313071250916
Epoch: 8, Batch: 0, Loss: 0.01611393690109253, Total Loss: 0.01611393690109253
Epoch: 9, Batch: 0, Loss: 0.0044406866654753685, Total Loss: 0.0044406866654753685
Epoch: 10, Batch: 0, Loss: 0.0016583320684731007, Total Loss: 0.0016583320684731007
Epoch: 11, Batch: 0, Loss: 0.0017648006323724985, Total Loss: 0.0017648006323724985
Epoch: 12, Batch: 0, Loss: 0.002

Epoch: 99, Batch: 0, Loss: 0.0003044720215257257, Total Loss: 0.0003044720215257257
Epoch: 100, Batch: 0, Loss: 0.00029752569389529526, Total Loss: 0.00029752569389529526
Epoch: 101, Batch: 0, Loss: 0.00011103472206741571, Total Loss: 0.00011103472206741571
Epoch: 102, Batch: 0, Loss: 0.00018468529742676765, Total Loss: 0.00018468529742676765
Epoch: 103, Batch: 0, Loss: 0.0006430173525586724, Total Loss: 0.0006430173525586724
Epoch: 104, Batch: 0, Loss: 0.0019401067402213812, Total Loss: 0.0019401067402213812
Epoch: 105, Batch: 0, Loss: 0.0011122183641418815, Total Loss: 0.0011122183641418815
Epoch: 106, Batch: 0, Loss: 0.00022784658358432353, Total Loss: 0.00022784658358432353
Epoch: 107, Batch: 0, Loss: 0.00010953752644127235, Total Loss: 0.00010953752644127235
Epoch: 108, Batch: 0, Loss: 0.0006609426927752793, Total Loss: 0.0006609426927752793
Epoch: 109, Batch: 0, Loss: 0.0011653536930680275, Total Loss: 0.0011653536930680275
Epoch: 110, Batch: 0, Loss: 9.761394903762266e-05, Total

#### Prepare Testing Data for Forecasting

In [ ]:
# Display testdata for forecasting
testdata.head(20)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-12-21,132.979996,136.809998,132.750000,135.449997,135.243500,85928000
1,2022-12-22,134.350006,134.559998,130.300003,132.229996,132.028412,77852100
2,2022-12-23,130.919998,132.419998,129.639999,131.860001,131.658981,63814900
3,2022-12-27,131.380005,131.410004,128.720001,130.029999,129.831772,69007800
4,2022-12-28,129.669998,131.029999,125.870003,126.040001,125.847855,85438400
5,2022-12-29,127.989998,130.479996,127.730003,129.610001,129.412415,75703700
6,2022-12-30,128.410004,129.949997,127.430000,129.929993,129.731918,77034200
7,2023-01-03,130.279999,130.899994,124.169998,125.070000,124.879326,112117500
8,2023-01-04,126.889999,128.660004,125.080002,126.360001,126.167366,89113600
9,2023-01-05,127.129997,127.769997,124.760002,125.019997,124.829399,80962700


#### Feature Scaling Testing Data

In [ ]:
# Feature scaling
min_max = (0,1)
# feature = "Open" #<= same as before

# Get the scaled Open
scale_test_open = MinMaxScaler(feature_range = min_max)
open_price = testdata[feature].values.reshape(-1, 1)
data_test_open = scale_test_open.fit_transform(open_price) # Scale the Open Price

# Get the dates
data_date = testdata.Date.values

#### Prepare Testing Data with Sliding Window

In [ ]:
# Extract data with sliding window
# Sequence_length same for train dataset
# year = "2023" # Same year as Training
# month = "01" # Same Month as Training


index_start_date = start_date_index(year, month, testdata)

test, actual, date, open_price = sliding_window_test(index_start_date, sequence_length, 
                                                     data_test_open, open_price, data_date)

Index of Date: 7
Start Date: 2023-01-03

Test shape: (76, 7, 1)

Test:
batch size: 76
sequence length: 7 days
input size: 1


Actual shape: (76, 1)

Actual:
batch size: 76
input size: 1



#### Prepare Testing Data for Forecasting

In [ ]:
# Set data to Tensor
test = Variable(torch.Tensor(np.array(test).reshape(-1,sequence_length,1)))
actual = Variable(torch.Tensor(np.array(actual).reshape(-1,1)))

### Model Prediction

In [ ]:
# Model prediction
lstm_model.eval()
with torch.no_grad():
    prediction = lstm_model(test.to(device))

#### Inverse Scale Prediction and Prepare for Visualization

In [ ]:
# Inverse scale transform
final_actual = scale_test_open.inverse_transform(actual)
final_prediction = scale_test_open.inverse_transform(prediction.cpu())

# Combine Actual with Predict
output_df = pd.DataFrame(data={"Date":date.reshape(-1,),
                               "Actual_Open":final_actual.reshape(-1,), 
                               "Predicted_Open":final_prediction.reshape(-1,)})
output_df

,Date,Actual_Open,Predicted_Open
0,2023-01-03,130.279999,129.136429
1,2023-01-04,126.889999,130.991425
2,2023-01-05,127.129997,128.414845
3,2023-01-06,126.010002,127.984885
4,2023-01-09,130.470001,126.974983
...,...,...,...
71,2023-04-17,165.089996,164.271827
72,2023-04-18,166.100006,165.214203
73,2023-04-19,165.800004,166.087307
74,2023-04-20,166.089997,165.905006


#### Calculate Model RMSE Loss

In [ ]:
# Calculate Model RMSE Loss
rmse_loss_np, mse_np = MSE_RMSE_loss_np(final_prediction, final_actual)

MSE Loss: 4.452670097351074 
RMSE Loss: 2.110135078430176

MSE Loss: 4.452668682285069 
RMSE Loss: 2.1101347545322953



#### Calculate Model R2

In [ ]:
r2_np = R2_np(final_prediction, final_actual)

R2: 0.9626431433358783

R2: 0.9626431433358783



### Save Model Forecast Results and Train Data Used

In [ ]:
if not os.path.exists("dataset"):
    os.mkdir("dataset")
output_df.to_csv(r"dataset\outputSingle.csv")
traindata.to_csv(r"dataset\traindataSingle.csv")